In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('FILES/hr_raw_data.csv')

In [ ]:
#AÑADIR ($) A NOMBRE DE LAS COLUMNAS: 
columnas_dolar = ["dailyrate", "hourlyrate", "monthlyincome", "monthlyrate", "sameasmonthlyincome", "salary"]
for c in columnas_dolar:
    df.rename(columns={c: c + "($)"}, inplace=True)



In [ ]:
#LIMPIEZA DE DATOS PARA CONVENTIR DE OBJECT A FLOAT
columnas_a_float = ["monthlyincome($)", "totalworkingyears", "salary($)", ]

for columna in columnas_a_float:

    df[columna] = df[columna].astype(str).str.replace('$', '', regex=False)
    print(f"La columna '{columna}' fue procesada para eliminación de símbolo $.")

    df[columna] = df[columna].str.replace(',', '.')
    print(f"La columna '{columna}' fue procesada para cambio de coma.")
    
    df[columna] = pd.to_numeric(df[columna], errors="coerce")
    print(f"La columna '{columna}' fue procesada para float.")


La columna 'monthlyincome($)' fue procesada para eliminación de símbolo $.
La columna 'monthlyincome($)' fue procesada para cambio de coma.
La columna 'monthlyincome($)' fue procesada para float.
La columna 'totalworkingyears' fue procesada para eliminación de símbolo $.
La columna 'totalworkingyears' fue procesada para cambio de coma.
La columna 'totalworkingyears' fue procesada para float.
La columna 'salary($)' fue procesada para eliminación de símbolo $.
La columna 'salary($)' fue procesada para cambio de coma.
La columna 'salary($)' fue procesada para float.


In [ ]:
#UNIFICAR Y LIMPIAR DATOS EN REMOTEWORK(BOOLEANO)
df["remotework"] = df["remotework"].replace({"Yes": True,
                                             "1": True,
                                             "0": False,
                                             "True": True,
                                             "False": False})

In [ ]:
#PONER TODOS LOS ELEMENTOS DE  LAS COLUMNAS EN MINÚSCULA
columnas_de_texto = df.select_dtypes(include=['object']).columns 

for columna in columnas_de_texto:
    df[columna] = df[columna].str.lower()
    

In [ ]:
#ELIMINAR LOS GUIONES BAJOS DE LOS VALORES DE LAS COLUMNAS
for columna in columnas_de_texto:
    df[columna] = df[columna].str.replace('_', ' ', regex=False)
    df[columna] = df[columna].str.replace('-', ' ', regex=False)

In [ ]:
#NORMALIZACIÓN MARITALSTATUS. 
df['maritalstatus'] = df['maritalstatus'].fillna('unknown').replace('marreid', 'married')


#DISTANCEFROMHOME
df[df['distancefromhome']<0].shape[0]
df['distancefromhome'].shape
porcentaje_negativos = (202/1678)*100 #CAMBIAR PARA QUE SEA VALIDO AUNQUE SE CAMBIEN LOS DATOS. 
print(f'porcentaje de negativos que contiene distancefromhome: {porcentaje_negativos}')
df['distancefromhome'] = abs(df['distancefromhome']) #CON ABS TODA LA SERIE FUNCIONA COMO UN VALOR ABSOLUTO
print(f'comprobamos cantidad de negativos en distancefromhome, tras conversion en valor absoluto: {df[df['distancefromhome']<0].shape[0]}')


porcentaje de negativos que contiene distancefromhome: 12.038140643623361
comprobamos cantidad de negativos en distancefromhome, tras conversion en valor absoluto: 0


In [ ]:
#NORMALIZACION DE AGE
mapeo = {'forty seven':47,'fifty eight':58, 'thirty six':36, 'fifty five':55,
       'fifty two':52, 'thirty one':31, 'thirty':30, 'twenty six':26, 'thirty seven':37,
       'thirty two':32, 'twenty four':24}


In [56]:
df["age"] = df["age"].replace(mapeo)

In [ ]:
#CONVERSION DE OBJECT A INT
columnas_a_convertir = ["age","worklifebalance", "performancerating", "numberchildren", "yearsincurrentrole"]
for i in df.index:
    for col in columnas_a_convertir:
        # valor original
        valor_original = df.loc[i, col]
        # reemplazar coma por punto
        valor = str(valor_original).replace(",", ".")
        try:
            # Intentar convertir a número
            numero = float(valor)
            df.loc[i, col] = int(numero)
        except:
            # dejar el valor tal cual
            df.loc[i, col] = valor_original
#  Convertir cada columna a entero nullable (Int64)
for col in columnas_a_convertir:
    df[col] = df[col].astype("Int64")